In [1]:
import pandas as pd
data = pd.read_csv("IMDB Dataset.csv")

In [ ]:
# data

In [2]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
lem = WordNetLemmatizer()
stemm = PorterStemmer()

In [5]:
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z0-9]', ' ', data['review'][i])
    review = review.lower()
    review = review.split()
#     review = [stemm.stem(word) for word in review if not word in stopwords.words('english')]
    review = [lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    if i == 99:
        break

In [6]:
len(corpus)

100

In [7]:
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

In [8]:
# applying sent_tokenization and then apply simple_preprocess 
# The gensim.utils.simple_preprocess function is a quick way to preprocess text by tokenizing it, 
# converting it to lowercase, and removing tokens that are too short or too long.

words=[]
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent_word in sent_token:
        words.append(simple_preprocess(sent))

### Word2Vec Model

In [9]:
import gensim
model = gensim.models.Word2Vec(words, window=5, min_count=2)

In [ ]:
# model.wv.index_to_key

In [ ]:
# model.corpus_count

In [ ]:
# model.epochs

In [ ]:
# model.wv.similar_by_word('maximum')

### AvgWord2Vec

In [10]:
import numpy as np
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)

In [11]:
from tqdm import tqdm
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 495.79it/s]


In [12]:
y_numpy = np.array([1 if label == 'positive' else 0 for label in data['sentiment']])[:100]

In [17]:
y_numpy

array([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1])

In [14]:
X_new = np.array(X)

In [18]:
# print(np.any(X_new < 0))  # True if there are negative values


True


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new,y_numpy, test_size=0.20, random_state=0)